In [1]:
import math, cmath
import numpy as np
np.set_printoptions(precision=2,floatmode='fixed',suppress=True)

class Observable:
    
    def __init__(self,e_val_1,e_vec_1,e_val_2,e_vec_2):
        self.e = (e_val_1,e_val_2)
        self.v = (np.array(e_vec_1),
                  np.array(e_vec_2))
        self.p = (self.outer(self.v[0]),self.outer(self.v[1]))
        self.m = self.e[0]*self.p[0]+self.e[1]*self.p[1]

    def outer(self,v):
        s = np.vdot(v,v)
        w = v.conjugate().reshape((2,1))
        return v*w/s  

    def qf(self,phi,m):
        s = np.vdot(phi,phi)
        phi = phi/cmath.sqrt(s)
        phi_p = np.dot(m,phi)
        return np.vdot(phi,phi_p)
    
    def probs(self,phi):
        return(self.qf(phi,self.v[0]),self.qf(phi,self.v[1]))

In [2]:
import numpy as np

# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi

In [3]:
# Create circuit
circ = QuantumCircuit(2)
circ.h(0)
circ.cx(0, 1)
circ.measure_all()
print(circ.draw(output='text'))

# Transpile for simulator
simulator = Aer.get_backend('aer_simulator')
circ = transpile(circ, simulator)

# Run and get counts
result = simulator.run(circ).result()
counts = result.get_counts(circ)
print(counts)

        ┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1 
{'11': 496, '00': 528}


In [4]:
import math, cmath
import numpy as np
np.set_printoptions(precision=2,floatmode='fixed',suppress=True)


def test_meas(make_circuit, calc_probability):
    simulator = Aer.get_backend('aer_simulator')
    #print(simulator.configuration().basis_gates)
    lambda_ = 0.0
    overall_d = 0.0
    for i in range(17):
        phi = i*math.pi/8.0
        print(f'\nphi: {i} pi/8\ntheta\tcalculated\texperimental')
        d = 0.0
        for j in range(17):
            theta = j*math.pi/16.0
            qc = make_circuit(simulator,theta,phi)
            result = simulator.run(qc,shots=1024).result()
            counts = result.get_counts(qc)
            pr_calc = calc_probability(theta,phi)
            if '0' not in counts:
                counts['0'] = 0
            if '1' not in counts:
                counts['1'] = 0 
            pr_expr = counts['0']/(counts['0']+counts['1'])
            print(f'{j} pi/16:\t{pr_calc:.4f}\t{pr_expr:.4f}')
            if abs(pr_expr-pr_calc)>d:
                d = abs(pr_expr-pr_calc)
        print(f'** max deviation: {d:.4f}')
        if d>overall_d:
            overall_d = d
    print(f'** overal max deviation: {overall_d:.4f}')


In [5]:
#Exercise A: I didn't change anything here

def z_probability(theta,phi):
    
    # this is the formula that we have been using.
    # it is for the z basis
    p = math.cos(theta/2.0)**2
    
    return p

def z_circuit(sim,theta,phi):
    qc = QuantumCircuit(1)
    # prepare test state
    qc.u(theta,phi,0,0)
    
    # measuring on the Z basis
    # need to put the correct gate here
        # qc.?()
        # no circuit needs to go here, 
        # qiskit measures already on the Z basis
    # 
    
    qc.measure_all()
    #print(qc.draw(output='text'))
    return transpile(qc,sim)


test_meas(z_circuit,z_probability)


phi: 0 pi/8
theta	calculated	experimental
0 pi/16:	1.0000	1.0000
1 pi/16:	0.9904	0.9932
2 pi/16:	0.9619	0.9561
3 pi/16:	0.9157	0.9248
4 pi/16:	0.8536	0.8623
5 pi/16:	0.7778	0.7607
6 pi/16:	0.6913	0.7119
7 pi/16:	0.5975	0.6514
8 pi/16:	0.5000	0.4980
9 pi/16:	0.4025	0.3984
10 pi/16:	0.3087	0.3145
11 pi/16:	0.2222	0.2031
12 pi/16:	0.1464	0.1377
13 pi/16:	0.0843	0.1016
14 pi/16:	0.0381	0.0439
15 pi/16:	0.0096	0.0098
16 pi/16:	0.0000	0.0000
** max deviation: 0.0538

phi: 1 pi/8
theta	calculated	experimental
0 pi/16:	1.0000	1.0000
1 pi/16:	0.9904	0.9961
2 pi/16:	0.9619	0.9658
3 pi/16:	0.9157	0.9102
4 pi/16:	0.8536	0.8428
5 pi/16:	0.7778	0.7793
6 pi/16:	0.6913	0.6689
7 pi/16:	0.5975	0.6250
8 pi/16:	0.5000	0.5059
9 pi/16:	0.4025	0.3984
10 pi/16:	0.3087	0.2969
11 pi/16:	0.2222	0.2305
12 pi/16:	0.1464	0.1602
13 pi/16:	0.0843	0.0879
14 pi/16:	0.0381	0.0381
15 pi/16:	0.0096	0.0068
16 pi/16:	0.0000	0.0000
** max deviation: 0.0275

phi: 2 pi/8
theta	calculated	experimental
0 pi/16:	1.0000	1.0000
1 

In [7]:
#Exercise B: X measurement

def x_probability(theta,phi):

    p =  (0.5)*(1.0+math.sin(theta)*math.cos(phi)) # need to put the correct formula here
    
    return p
    

def x_circuit(sim,theta,phi):
    qc = QuantumCircuit(1)
    # prepare test state
    qc.u(theta,phi,0,0)
    

    
    # measuring on the X basis
    # need to put the correct gate here
        # qc.?()
    qc.h(0) 

    
    qc.measure_all()

    return transpile(qc,sim)

test_meas(x_circuit,x_probability)


phi: 0 pi/8
theta	calculated	experimental
0 pi/16:	0.5000	0.5020
1 pi/16:	0.5975	0.5811
2 pi/16:	0.6913	0.7148
3 pi/16:	0.7778	0.7764
4 pi/16:	0.8536	0.8447
5 pi/16:	0.9157	0.9121
6 pi/16:	0.9619	0.9590
7 pi/16:	0.9904	0.9893
8 pi/16:	1.0000	1.0000
9 pi/16:	0.9904	0.9951
10 pi/16:	0.9619	0.9590
11 pi/16:	0.9157	0.9043
12 pi/16:	0.8536	0.8389
13 pi/16:	0.7778	0.7744
14 pi/16:	0.6913	0.7119
15 pi/16:	0.5975	0.5889
16 pi/16:	0.5000	0.4746
** max deviation: 0.0254

phi: 1 pi/8
theta	calculated	experimental
0 pi/16:	0.5000	0.4688
1 pi/16:	0.5901	0.5938
2 pi/16:	0.6768	0.6748
3 pi/16:	0.7566	0.7686
4 pi/16:	0.8266	0.8320
5 pi/16:	0.8841	0.8984
6 pi/16:	0.9268	0.9297
7 pi/16:	0.9531	0.9482
8 pi/16:	0.9619	0.9551
9 pi/16:	0.9531	0.9590
10 pi/16:	0.9268	0.9316
11 pi/16:	0.8841	0.8945
12 pi/16:	0.8266	0.8174
13 pi/16:	0.7566	0.7520
14 pi/16:	0.6768	0.6709
15 pi/16:	0.5901	0.5850
16 pi/16:	0.5000	0.4697
** max deviation: 0.0312

phi: 2 pi/8
theta	calculated	experimental
0 pi/16:	0.5000	0.4717
1 

In [12]:
def y_probability(theta,phi):
    
    p = (0.5)*(1.0+math.sin(theta)*math.sin(phi)) # need to put the correct formula here
    
    return p


def y_circuit(sim,theta,phi):
    qc = QuantumCircuit(1)
    qc.u(theta,phi,0,0)
    
    # measuring on the Y basis
    # need to put the correct gate here
        # qc.?()
    # 
    qc.u(math.pi/2, math.pi/2, math.pi/2,0)

    qc.measure_all()
    return transpile(qc,sim)

test_meas(y_circuit,y_probability)


phi: 0 pi/8
theta	calculated	experimental
0 pi/16:	0.5000	0.4902
1 pi/16:	0.5000	0.5049
2 pi/16:	0.5000	0.4951
3 pi/16:	0.5000	0.4971
4 pi/16:	0.5000	0.5137
5 pi/16:	0.5000	0.5264
6 pi/16:	0.5000	0.5137
7 pi/16:	0.5000	0.5117
8 pi/16:	0.5000	0.5205
9 pi/16:	0.5000	0.5117
10 pi/16:	0.5000	0.4912
11 pi/16:	0.5000	0.4941
12 pi/16:	0.5000	0.5049
13 pi/16:	0.5000	0.4990
14 pi/16:	0.5000	0.4951
15 pi/16:	0.5000	0.4922
16 pi/16:	0.5000	0.5312
** max deviation: 0.0312

phi: 1 pi/8
theta	calculated	experimental
0 pi/16:	0.5000	0.5391
1 pi/16:	0.5373	0.5244
2 pi/16:	0.5732	0.5723
3 pi/16:	0.6063	0.5967
4 pi/16:	0.6353	0.6348
5 pi/16:	0.6591	0.6895
6 pi/16:	0.6768	0.6709
7 pi/16:	0.6877	0.6895
8 pi/16:	0.6913	0.6768
9 pi/16:	0.6877	0.6836
10 pi/16:	0.6768	0.6816
11 pi/16:	0.6591	0.6562
12 pi/16:	0.6353	0.6279
13 pi/16:	0.6063	0.6055
14 pi/16:	0.5732	0.5352
15 pi/16:	0.5373	0.5234
16 pi/16:	0.5000	0.5215
** max deviation: 0.0391

phi: 2 pi/8
theta	calculated	experimental
0 pi/16:	0.5000	0.5117
1 